In [1]:
from scipy import io

In [2]:
mat = io.loadmat('data/p300S01.mat')

In [3]:
data = mat['data']
data

array([[(array([[26.95640373, 11.73869324,  6.98503304, ...,  2.66381407,
                -3.04957318, -2.63618731],
               [29.94702721, 14.98267174, 10.02626705, ...,  5.16736984,
                 1.30663943,  2.16391516],
               [33.09702683, 19.32771683, 13.60309792, ...,  8.05283546,
                 8.53473663,  7.90571976],
               ...,
               [ 5.6236558 ,  0.53022534,  1.55959177, ..., -4.96036482,
                 2.35364461, -8.1143961 ],
               [ 4.38928461,  1.46526659,  1.96254373, ..., -4.6057415 ,
                 1.29674733, -7.18669939],
               [ 2.58838463,  0.79275906,  0.29564747, ..., -4.98137999,
                -4.04607677, -7.54975986]]), array([[0],
               [0],
               [0],
               ...,
               [0],
               [0],
               [0]], dtype=uint8), array([[0],
               [0],
               [0],
               ...,
               [0],
               [0],
               [0]], d

In [4]:
data['flash'][0][0]

array([[  7486,     31,     11,      1],
       [  7548,     31,      5,      1],
       [  7610,     30,      7,      1],
       ...,
       [355688,     32,      3,      2],
       [355750,     32,      7,      1],
       [355813,     32,      4,      1]])